# Exercise 1: Building a Baseline Model

In this exercise, you will develop a baseline model for predicting if a customer will buy the app after clicking through from an ad. With this baseline model, you'll be able to see how your feature engineering and selection efforts improve the model's performance.

In [ ]:
# Set up code checking
from learntools.core import binder
binder.bind(globals())
from learntools.machine_learning.ex2 import *

In [ ]:
import pandas as pd

click_data = pd.read_csv('../input/train_sample.csv', parse_dates=['click_time'])
click_data.head(10)

## Baseline Model

The first thing you need to do is construct a baseline model. All new features, processing, encodings, and feature selection should improve upon this baseline model. First you need to do a bit of feature engineering before training the model itself.

### 1) Exercise: Features from timestamps
From the timestamps, create features for the day, hour, minute and second. Store these as new integer columns `day`, `hour`, `minute`, and `second` and a new DataFrame `clicks`.

In [ ]:
# Add new columns for timestamp features day, hour, minute, and second
clicks = ____

In [ ]:
#%%RM_IF(PROD)%%
# My solution, there are a lot of ways to do this

# Split up the times
click_times = click_data['click_time']
clicks = click_data.assign(day=click_times.dt.day.astype('uint8'),
                           hour=click_times.dt.hour.astype('uint8'),
                           minute=click_times.dt.minute.astype('uint8'),
                           second=click_times.dt.second.astype('uint8'))

In [ ]:
clicks.head()

### 2) Exercise: Label Encoding
For each of the categorical features `['ip', 'app', 'device', 'os', 'channel']`, use scikit-learn's `LabelEncoder` to create new features in the `clicks` DataFrame. The new column names should be the original column name with `'_labels'` appended, like `ip_labels`.

In [ ]:
from sklearn import preprocessing

In [ ]:
cat_features = ['ip', 'app', 'device', 'os', 'channel']

# Create new columns in using preprocessing.LabelEncoder()
____


In [ ]:
#%%RM_IF(PROD)%%

# Label encode categorical variables 
cat_features = ['ip', 'app', 'device', 'os', 'channel']
for feature in cat_features:
    label_encoder = preprocessing.LabelEncoder()
    new_feature = label_encoder.fit_transform(clicks[feature])
    clicks[feature +'_labels'] = new_feature

In [ ]:
clicks.head()

### 3) One-hot encoding?

Now you have label encoded features, does it make sense to use one-hot encoding at this point?

**Answer:** the `ip_labels` column has 58,000 values, which means it will create an extremely sparse matrix with 58,000 columns. Generally a bad idea. Luckily, LightGBM works well with label encoded features.

## Train, validation, and test sets
With our baseline features ready, we need to split our data into training and validation sets. We should also hold out a test set to measure the final accuracy of the model.

### 4) Question, train/test splits with time series data
This is time series data. Are they any special considerations when creating train/test splits for time series? If so, what and why?

**Answer:** Since our model is meant to predict events in the future, we must also validate the model on events in the future. If the data is mixed up between the training and test sets, then future data will leak in to the model and our validation results will overestimate the performance on new data.

### 5) Exercise: Create train/validation/test splits

First, sort the data in order of increasing time. Use the first 80% of the rows as the train set, the next 10% as the validation set, and the last 10% as the test set. Then create LightGBM Dataset objects for each of the smaller datasets.

In [ ]:
import lightgbm as lgb

In [ ]:
feature_cols = ['day', 'hour', 'minute', 'second', 
                'ip_labels', 'app_labels', 'device_labels',
                'os_labels', 'channel_labels']

# Be sure to sort the data by click time before making the splits
# Your code here: create train, validation, and test sets as lgb.Dataset objects
dtrain = ____
dvalid = ____
dtest = ____

In [ ]:
#%%RM_IF(PROD)%%

feature_cols = ['day', 'hour', 'minute', 'second', 
                'ip_labels', 'app_labels', 'device_labels',
                'os_labels', 'channel_labels']


valid_fraction = 0.1
clicks = clicks.sort_values('click_time')
valid_rows = int(len(clicks) * valid_fraction)
train = clicks[:-valid_rows * 2]
# valid size == test size, last two sections of the data
valid = clicks[-valid_rows * 2:-valid_rows]
test = clicks[-valid_rows:]


dtrain = lgb.Dataset(train[feature_cols], label=train['is_attributed'])
dvalid = lgb.Dataset(valid[feature_cols], label=valid['is_attributed'])
dtest = lgb.Dataset(test[feature_cols], label=test['is_attributed'])

In [ ]:
param = {'num_leaves': 64, 'objective': 'binary'}
param['metric'] = 'auc'
num_round = 1000
bst = lgb.train(param, dtrain, num_round, valid_sets=[dvalid], early_stopping_rounds=10)

### 6) Exercise: Evaluate the model
Finally, with the model trained, evaluate it's performance on the test set `dtest`. Use the ROC AUC score from `sklearn.metrics`.

In [ ]:
from sklearn import metrics

In [ ]:
# Your code here. Make predictions on the test set with the trained LightGBM model
ypred = ____ 
score = ____

In [ ]:
#%%RM_IF(PROD)%%
ypred = bst.predict(test[feature_cols])
score = metrics.roc_auc_score(test['is_attributed'], ypred)

In [ ]:
print(f"Test score: {score}")

This will be our baseline score for the model. When we transform features, add new ones, or perform feature selection, we should be improving on this score.